# Line RT: 3D homogeneous LTE sphere, constant velocity

## 0) Setup

In [1]:
# Define Magritte folders

MagritteSetupFolder = '/home/frederik/Dropbox/Astro/Magritte/modules/setup/'
ProjectFolder       = '/home/frederik/MagritteProjects/Lines_3D_LTE/'

In [2]:
# Import tools and libraries

import numpy as np

from bokeh.plotting import figure, show, gridplot
from bokeh.palettes import cividis
from bokeh.io       import output_notebook
output_notebook()

from glob import glob
from sys  import path
path.insert(0, MagritteSetupFolder)

# Import from MagritteSetupFolder
from spheres  import deSphere, sphereVar, mapTo3D
from lineData import LineData, planck, relativeDifference
from setup    import setupMagritte
from model    import model

Loading BokehJS ...

## 1) Define model

Assume a 1D model (i.e. 1 ray) with constant temperature, density and abundances and a linear velocity field, yielding a constant velocity gradient

\begin{align}
T(r)     \ &= \ cte, \\
\rho(r)  \ &= \ cte, \\
n_{i}(r) \ &= \ cte, \\
v(r)     \ &= \ cte,
\end{align}

where $L$ is the total length of the model and $v_{\max}$ is the maximum speed reached in the model. (Numerical values given below.) We write it in this way to ensure that velocities in the model are always much smaller than $c$, since our equations for Doppler shift are not valid otherwise.

There is CMB radiation incoming on both ends of the ray as boundary condition.

In [3]:
# Define model

model = model (dim=1)

model.ncells = 23

dens = 1.0E+12   # [m^-3]
abun = 1.0E+4    # [m^-3]
temp = 1.0E+02   # [K]
dx   = 1.0E5     # [m]
dT   = 1.0E1     # [K]
dv   = 0.0E0     # [m/s]

model.density     = [dens     for _ in range(model.ncells)]
model.abundance   = [abun     for _ in range(model.ncells)]
model.temperature = [temp     for _ in range(model.ncells)]

model.x = [dx*i  for i in range(model.ncells)]
model.y = [0.0   for _ in range(model.ncells)]
model.z = [0.0   for _ in range(model.ncells)]

model.vx = [0.0 for _ in range(model.ncells)]
model.vy = [0.0 for _ in range(model.ncells)]
model.vz = [0.0 for _ in range(model.ncells)]

model.boundary = [0, model.ncells-1]

# Define nsidesList for mapping to 3D model
nsidesList = [int(i/3) for i in range(model.ncells)]

# Map 1D model to spherically symmetric 3D model
(model3D, cellsInShell) = mapTo3D(model, nsidesList)

# Define rays
model3D.defineRays (nsides=3)

# Write new model data
model3D.writeInput (ProjectFolder + 'Magritte_files/')

# Run setup
setupMagritte (projectFolder = ProjectFolder, runName = '')

print(f'ncells in 3D model = {model3D.ncells}')
print(f'nrays  in 3D model = {model3D.nrays} ')

ncells in 3D model = 4455
nrays  in 3D model = 108 


In [4]:
# Plot model

plot_model_1 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_1.line (model.x, model.density)
plot_model_1.xaxis.axis_label = "x [m]"
plot_model_1.yaxis.axis_label = "density [m^-3]"

plot_model_2 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_2.line (model.x, model.abundance)
plot_model_2.xaxis.axis_label = "x [m]"
plot_model_2.yaxis.axis_label = "abundance [xm^-3]"

plot_model_3 = figure(plot_width=400, plot_height=250)
plot_model_3.line (model.x, model.temperature)
plot_model_3.xaxis.axis_label = "x [m]"
plot_model_3.yaxis.axis_label = "temperature [K]"

plot_model_4 = figure(plot_width=400, plot_height=250)
plot_model_4.line (model.x, model.vx)
plot_model_4.xaxis.axis_label = "x [m]"
plot_model_4.yaxis.axis_label = "velocity_x [m^-3]"

plot_model = gridplot ([[plot_model_1, plot_model_2],[plot_model_3, plot_model_4]])

show (plot_model)

In [5]:
# Interactive 3D plot

from IPython.core.display import display, HTML
from string import Template
import json

In [6]:
%%html
<div id="mynetwork"></div>

In [7]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    var nodes = [
        {id: 1, label: 'Beyonce', group: 'United States'},
        {id: 2, label: 'Barak Obama', group: 'United States'},
        {id: 3, label: 'Miley Cyrus', group: 'United States'},
        {id: 4, label: 'Pope Francis', group: 'Vatican'},
        {id: 5, label: 'Vladimir Putin', group: 'Rusia'}
    ];

    // create an array with edges
    var edges = [
        {from: 1, to: 2},
        {from: 1, to: 3},
        {from: 2, to: 4},
        {from: 2, to: 5}
    ];

    // create a network
    var container = document.getElementById('mynetwork');
    var data= {
        nodes: nodes,
        edges: edges,
    };
    var options = {
        width: '800px',
        height: '400px'
    };
    
    var network = new vis.Network(container, data, options);
});

<IPython.core.display.Javascript object>

### Read Magritte output

In [5]:
# Read Magritte output

ioFolders = glob(ProjectFolder + 'io/*/')
ioFolders.sort()

inputFolders  = [folder +  'input/' for folder in ioFolders]
outputFolders = [folder + 'output/' for folder in ioFolders]

lastOutput = outputFolders[-1]
lastInput  =  inputFolders[-1]

pops_files = glob(lastOutput + 'populations_0*.txt')
Jeff_files = glob(lastOutput + 'Jeff_0*.txt')
J_files    = glob(lastOutput + 'J_*.txt')
nu_files   = glob(lastOutput + 'frequencies_nu*.txt')
lnr_files  = glob(lastOutput + 'frequencies_line_nr*.txt')
eta_files  = glob(lastOutput + 'eta_0*.txt')
chi_files  = glob(lastOutput + 'chi_0*.txt')

pops_files.sort()
Jeff_files.sort()
eta_files.sort()
chi_files.sort()

pops_3D_data = [np.loadtxt(fileName) for fileName in pops_files]
Jeff_3D_data = [np.loadtxt(fileName) for fileName in Jeff_files]
J_3D_data    = [np.loadtxt(fileName) for fileName in J_files]
nu_3D_data   = [np.loadtxt(fileName) for fileName in nu_files]
lnr_3D_data  = [np.loadtxt(fileName) for fileName in lnr_files]
eta_3D_data  = [np.loadtxt(fileName) for fileName in eta_files]
chi_3D_data  = [np.loadtxt(fileName) for fileName in chi_files]

# Import linedata
lineData = LineData (ProjectFolder + '/Magritte_files/linedata/hco+.dat')

### Map sphere to 1D model

In [6]:
pops_data = deSphere(cellsInShell, pops_3D_data)
pops_var  = sphereVar(cellsInShell, pops_3D_data)

J_data = deSphere(cellsInShell, J_3D_data)
J_var  = sphereVar(cellsInShell, J_3D_data)

Jeff_data = deSphere(cellsInShell, Jeff_3D_data)
Jeff_var  = sphereVar(cellsInShell, Jeff_3D_data)

nu_data = deSphere(cellsInShell, nu_3D_data)
nu_var  = sphereVar(cellsInShell, nu_3D_data)

lnr_data = deSphere(cellsInShell, lnr_3D_data)
lnr_var  = sphereVar(cellsInShell, lnr_3D_data)

#freqsR   = deSphere(cellsInShell, [freqs])
#freqsVar = sphereVar(cellsInShell, [freqs])

#etaR   = deSphere(cellsInShell, eta_data)
#etaVar = sphereVar(cellsInShell, eta_data)

#chiR   = deSphere(cellsInShell, chi_data)
#chiVar = sphereVar(cellsInShell, chi_data)

for s in range(model.ncells):
    print(f'pops_var  = {np.max(pops_var[0][s])}')
    print(f'Jeff_var  = {np.max(Jeff_var[0][s])}')
    print(f'J_var     = {np.max(J_var[0][s])}')
    print(f'nu_var    = {np.max(nu_var[0][s])}')
    print(f'lnr_var   = {np.max(lnr_var[0][s])}')

#print(np.max(etaVar))
#print(np.max(chiVar))

pops_var  = 0.0
Jeff_var  = 0.0
J_var     = 0.0
nu_var    = 0.0
lnr_var   = 0.0
pops_var  = 0.0
Jeff_var  = 0.0
J_var     = 0.0
nu_var    = 0.0
lnr_var   = 0.0
pops_var  = 0.0
Jeff_var  = 0.0
J_var     = 0.0
nu_var    = 0.0
lnr_var   = 0.0
pops_var  = 1.8251018876176273e-16
Jeff_var  = 0.0
J_var     = 0.01723429602603357
nu_var    = 2.056052358372553e-16
lnr_var   = 0.0
pops_var  = 1.8251018876176273e-16
Jeff_var  = 0.0
J_var     = 0.009496585194363001
nu_var    = 2.056052358372553e-16
lnr_var   = 0.0
pops_var  = 1.8251018876176273e-16
Jeff_var  = 0.0
J_var     = 0.01872904291449999
nu_var    = 2.056052358372553e-16
lnr_var   = 0.0
pops_var  = 6.182604716840357e-16
Jeff_var  = 0.0
J_var     = 0.02583811174366245
nu_var    = 6.845012688035688e-16
lnr_var   = 0.0
pops_var  = 6.182604716840357e-16
Jeff_var  = 0.0
J_var     = 0.019155290819921258
nu_var    = 6.845012688035688e-16
lnr_var   = 0.0
pops_var  = 6.182604716840357e-16
Jeff_var  = 0.0
J_var     = 0.011023476060447403
nu_var    = 

### Plot output

In [9]:
s_min  = 0
s_max  = model.ncells
s_step = 1
ns     = int((s_max - s_min) / s_step)

In [10]:
plot_pops = figure (title='Level populations', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    plot_pops.line(range(lineData.nlev), pops_data[0][s], color=cividis(ns)[int(s/s_step)], legend=str(s))
plot_pops.xaxis.axis_label = "number of the level"
plot_pops.yaxis.axis_label = "population [m^-3]"
show (plot_pops)

In [11]:
plot_Jeff = figure (title='Total mean intensity', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    plot_Jeff.line(range(lineData.nrad), Jeff_data[1][s], color=cividis(ns)[int(s/s_step)], legend=str(s))
plot_Jeff.xaxis.axis_label = "number of the transition"
plot_Jeff.yaxis.axis_label = "mean intensity J [m^-3]"
show (plot_Jeff)

In [12]:
plot_spectrum = figure (title='Spectrum', width=700, height=500, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    plot_spectrum.line(nu_data[0][s], J_data[0][s], color=cividis(ns)[int(s/s_step)], legend=str(s))
plot_spectrum.xaxis.axis_label = "frequencies [Hz]"
plot_spectrum.yaxis.axis_label = "Mean intensity [W/m^2]"
show(plot_spectrum)

## Analytical solution

Assuming a constant source function $S_{\nu}(x)=S_{\nu}$ along the ray and boundary condition $B_{\nu}$ on both sides of the ray, the mean intensity is given by

\begin{align}
    J_{\nu}^{\text{ray}}(\tau(x)) \ = \ S_{\nu} \ + \ \frac{1}{2} \ \left(B_{\nu}-S_{\nu}\right) \ \left[e^{-\tau_{\nu}(x)} + e^{-\tau_{\nu}(L-x)}\right],
\end{align}

where the optical depth $\tau_{\nu}$ is given by

\begin{equation}
    \tau_{\nu}(\ell) \ = \ \int_{0}^{\ell} \text{d} l \ \chi_{\nu}(l) .
\end{equation}

The frequency dependence of the opacity only comes from the line profile

\begin{equation}
    \chi_{\nu}(x) \ = \ \chi_{ij} \phi_{\nu},
\end{equation}

where we assume a Gaussian profile

\begin{equation}
	\phi_{\nu}^{ij}(x) \ = \ \frac{1}{\sqrt{\pi} \ \delta\nu_{ij}} \ \exp \left[-\left(\frac{\nu-\nu_{ij}} {\delta\nu_{ij}(x)}\right)^{2}\right], \hspace{5mm} \text{where} \hspace{5mm} \delta\nu_{ij}(x) \ = \ \frac{\nu_{ij}}{c} \sqrt{ \frac{2 k_{b} T(x)}{m_{\text{spec}}} \ + \ v_{\text{turb}}^{2}(x)}.
\end{equation}

In a sphere of radius $R$, on a dinstence $r$ from the center, the distance $z$ to the boundary in a direction under an angle $\theta$ with the position vector $\textbf{r}$ is

\begin{equation}
z(r,\theta) \ = \ \sqrt{R^{2} + r^{2} \left(1 + \sin^{2} \theta\right)} - r \sin \theta .
\end{equation}
so the optical depth along such a ray is

\begin{equation}
    \tau_{\nu}(r,\theta) \ = \ \chi_{\nu}(l) z(r,\theta)
\end{equation}

and hence the mean intensity is

\begin{align}
    J_{\nu}(r) \ = \ S_{\nu} \ + \ \left(B_{\nu}-S_{\nu}\right) \ \int_{0}^{\pi} \frac{\text{d} \theta}{\pi} \ e^{-\tau_{\nu}(r,\theta)},
\end{align}

In [13]:
import scipy.integrate as integrate

c     = 2.99792458E+8    # [m/s] speed of light
kb    = 1.38064852E-23   # [J/K] Boltzmann's constant
mp    = 1.6726219E-27    # [kg]  proton mass
T_CMB = 2.7254800        # [K]   CMB temperature
vturb = 0.12012E3        # [m/s] turbulent speed

inverse_sqrt_pi = 0.5641895835478

pops       = lineData.LTEpop(temp) * abun
emissivity = lineData.lineEmissivity(pops)
opacity    = lineData.lineOpacity(pops)
source     = emissivity / opacity
bcd        = planck(T_CMB, np.array(lineData.frequency))

line = 19

S    =  source[line]
B    =     bcd[line]
chi  = opacity[line]
R    = model.x[-1]
vmax = model.vx[-1]
nuij = lineData.frequency[line]
dnu  = nuij/c * np.sqrt(2.0*kb*temp/mp + vturb**2)

def phi(nu):
    return inverse_sqrt_pi / dnu * np.exp(-((nu-nuij)/dnu)**2)

def z(r, theta):
    return np.sqrt(R**2 + r**2 * (1 + np.sin(theta)**2)) - r*np.sin(theta)

def J(nu, r):
    return S + (B-S) / np.pi * integrate.quad (lambda theta: np.exp(-chi*phi(nu)*z(r,theta)), 0.0, np.pi)[0]

def relativeError(a,b):
    return 2.0 * np.abs((a-b)/(a+b))

In [18]:
s_min = 0
s_max = model.ncells

plot_line_model = figure(title='Line model: freq dependence', width=450, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    # model
    vv = nuij + 8 * dnu * np.linspace(-1,1,500)
    xx = model.x[s]
    JJ = [J(v,xx) for v in vv]
    plot_line_model.line(vv, JJ)
    # data
    M = int(lnr_data[0][s][line] - 8    )
    N = int(lnr_data[0][s][line] + 8 + 1)
    plot_line_model.circle(nu_data[0][s][M:N], J_data[0][s][M:N], color=cividis(ns)[int(s/s_step)], legend=str(s))

plot_line_error = figure(title='Line error', width=450, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 8    )
    N = int(lnr_data[0][s][line] + 8 + 1)
    vv = nu_data[0][s][M:N]
    xx = model.x[s]
    JJ = [J(v,xx) for v in vv]
    error = relativeError(JJ, J_data[0][s][M:N])
    plot_line_error.circle(vv, error, color=cividis(ns)[int(s/s_step)], legend=str(s))

plot_line = gridplot([[plot_line_model, plot_line_error]])

show (plot_line)

In [25]:
plot_line_model = figure(title='Line model: space dependence', width=450, height=400, y_axis_type="log")
#for s in range(s_min, s_max, 2):
# model
xx = np.array(model.x)
plot_line_model.line(xx, J(nuij,xx))
# data
M = int(lnr_data[0][0][line])
plot_line_model.circle(xx, J_data[0][:,M])
plot_line_model.xaxis.axis_label = "x [m]"
plot_line_model.yaxis.axis_label = "Mean intensity [W/m^2]"

plot_line = gridplot([[plot_line_model, plot_line_error]])

show (plot_line)

In [26]:
plot_line_model = figure(title='Line model', width=450, height=400, y_axis_type="log")
#for s in range(s_min, s_max, 2):
# model
xx = np.array(model.x)
plot_line_model.line(xx, J(nuij,xx))
# data
M = int(lnr_data[0][0][line])
plot_line_model.circle(xx, Jeff_data[0][:,line])

plot_line = gridplot([[plot_line_model, plot_line_error]])

show (plot_line)

In [7]:
for a in range(-4):
    print(a)